
# リカレント ニューラル ネットワーク

:ラベル: `chap_rnn`

これまでは主に固定長データに焦点を当ててきました。 :numref: `chap_regression`および :numref: `chap_classification`で線形回帰とロジスティック回帰を導入し、 :numref: `chap_perceptrons`で多層パーセプトロンを導入するとき、各特徴ベクトル $\mathbf{x}_i$ が固定数の成分で構成されていると想定して問題ありませんでした。 x_1、\dots、x_d$。各数値特徴 $x_j$ は特定の属性に対応します。これらのデータセットは*テーブルに配置できるため、テーブル形式*と呼ばれることもあります。各サンプル $i$ は独自の行を取得し、各属性は独自の列を取得します。重要なのは、表形式データでは、列に対して特定の構造を想定することはほとんどありません。

続いて、 :numref: `chap_cnn`では、入力が画像内の各座標の生のピクセル値で構成される画像データに移りました。画像データは、典型的な表形式のデータセットの要件にほとんど適合しません。そこでは、階層構造と不変性を処理するために畳み込みニューラル ネットワーク (CNN) を呼び出す必要がありました。ただし、データは依然として固定長でした。すべての Fashion-MNIST 画像は、ピクセル値の $28 \times 28$ グリッドとして表されます。さらに、私たちの目標は、1 つの画像だけを調べて 1 つの予測を出力するモデルを開発することでした。しかし、ビデオなどの一連の画像に直面した場合、または画像キャプションの場合のように、連続的に構造化された予測を生成するというタスクを課された場合、どうすればよいでしょうか?

数え切れないほどの学習タスクでは、連続データを処理する必要があります。画像キャプション、音声合成、音楽生成ではすべて、モデルがシーケンスで構成される出力を生成する必要があります。時系列予測、ビデオ分析、音楽情報検索などの他の分野では、モデルはシーケンスである入力から学習する必要があります。これらの要求は同時に発生することがよくあります。テキストの一節をある自然言語から別の自然言語に翻訳する、対話を行う、ロボットを制御するなどのタスクでは、モデルが順次構造化されたデータの取り込みと出力の両方を行う必要があります。

リカレント ニューラル ネットワーク (RNN) は、ノードのネットワーク内のサイクルと考えることができる、*リカレント*接続を介してシーケンスのダイナミクスをキャプチャする深層学習モデルです。これは最初は直観に反するように思えるかもしれません。結局のところ、計算の順序を明確にするのは、ニューラル ネットワークのフィードフォワードの性質です。ただし、反復エッジは正確な方法で定義されているため、そのような曖昧さは発生しません。リカレント ニューラル ネットワークは、各ステップに適用される*同じ*基礎パラメーターを使用して、タイム ステップ (またはシーケンス ステップ) 全体に*展開されます*。標準接続は*同期的に*適用され、各層のアクティベーションを*同じタイム ステップで*後続の層に伝播しますが、再帰接続は*動的で*あり、隣接するタイム ステップ間で情報を渡します。 :numref: `fig_unfolded-rnn`の展開図からわかるように、RNN は、各層のパラメーター (従来型と再帰型の両方) がタイム ステップ全体で共有されるフィードフォワード ニューラル ネットワークと考えることができます。 

![](../img/unfolded-rnn.svg) :label: `fig_unfolded-rnn`

より広範なニューラル ネットワークと同様に、RNN には分野にわたる長い歴史があり、認知科学者によって普及された脳のモデルとして始まり、その後機械学習コミュニティで使用される実用的なモデリング ツールとして採用されました。より広範な深層学習と同様に、本書は機械学習の観点を採用しており、手書き認識 :cite: `graves2008novel` 、機械翻訳 :cite: `Sutskever.Vinyals.Le.2014`のさまざまなタスクにおける画期的な結果により、2010 年代に人気が高まった実用的なツールとしての RNN に焦点を当てています。 `Sutskever.Vinyals.Le.2014` 、および医療診断の認識:cite: `Lipton.Kale.2016` 。さらに詳しい背景資料に興味のある読者には、公開されている包括的なレビューを参照してください:cite: `Lipton.Berkowitz.Elkan.2015` 。また、順序性は RNN に固有のものではないことにも注意してください。たとえば、すでに導入した CNN は、さまざまな長さのデータ、たとえばさまざまな解像度の画像を処理するように適合させることができます。さらに、RNN は最近、かなりの市場シェアを Transformer モデルに譲ってきました。これについては、 :numref: `chap_attention-and-transformers`で説明します。ただし、RNN は、深層学習で複雑な逐次構造を処理するためのデフォルトのモデルとして注目を集め、今日に至るまで逐次モデリングの主要なモデルであり続けています。 RNN とシーケンス モデリングの話は密接に関連しており、これは RNN に関する章であると同時に、シーケンス モデリングの問題の ABC についての章でもあります。

 1 つの重要な洞察が、シーケンス モデリングにおける革命への道を切り開きました。機械学習の多くの基本的なタスクの入力とターゲットは、固定長ベクトルとして簡単に表現できませんが、多くの場合、固定長ベクトルの可変長シーケンスとして表現できます。たとえば、文書は一連の単語として表現できます。医療記録は、多くの場合、一連のイベント (診察、投薬、処置、臨床検査、診断) として表すことができます。ビデオは、可変長の静止画像シーケンスとして表現できます。

シーケンス モデルは数え切れないほどの応用分野で登場していますが、この分野の基礎研究は主に自然言語処理の中核タスクの進歩によって推進されてきました。したがって、この章ではテキスト データに関する説明と例に焦点を当てます。これらの例を理解すれば、これらのモデルを他のデータ モダリティに適用することは比較的簡単になるはずです。次のいくつかのセクションでは、シーケンスの基本的な表記法と、シーケンシャルに構造化されたモデル出力の品質を評価するためのいくつかの評価尺度を紹介します。次に、言語モデルの基本概念について説明し、この議論を最初の RNN モデルの動機付けに使用します。最後に、RNN を介して逆伝播するときに勾配を計算する方法について説明し、そのようなネットワークをトレーニングするときによく遭遇するいくつかの課題を検討し、 :numref: `chap_modern_rnn`に続く最新の RNN アーキテクチャの動機付けとなります。

 :begin_tab:toc
- [順序](sequence.ipynb)
- [テキストシーケンス](text-sequence.ipynb)
- [言語モデル](language-model.ipynb)
- [ん](rnn.ipynb)
- [rnn-スクラッチ](rnn-scratch.ipynb)
- [rnn-簡潔](rnn-concise.ipynb)
- [bptt](bptt.ipynb) :end_tab:
